## Demo for generating PSF model at given imaging depth

#### Setup environment

In [1]:
import sys
sys.path.append("..")
from psflearning.psflearninglib import psflearninglib
from psflearning import io
from psflearning.makeplots import *
import tensorflow as tf
#main_data_dir = io.param.load('datapath.yaml').main_data_dir
try:
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print('Running on GPU')
except:
    print('Running on CPU')

Running on GPU


#### Load learned PSF model from bead data

In [45]:
resfile = r'E:\EMBL files\data for PSF learning\example data for uiPSF\1ch_FD_smlm_nup96\psfmodel_20_insitu_FD_single.h5'
#resfile = main_data_dir+'results/2ch_40nm_bead/psfmodel_zernike_vector_multi.h5'
f,p = io.h5.load(resfile) # load result file
L = psflearninglib()
L.param = p
#L.param.PSFtype = 'insitu_zernike'

#### Generate PSF model at a given stage position
`Nz`: number of z slices  
`xsz`: number of pixels in both x and y dimension  
`stagepos`: stage position, value should be always positive, unit: micron  
Note: other parameters can be modified using `L.param`, such as pixel size, NA, refractive indices and wavelength

In [46]:
Nz = 21
xsz = 21
f,psfobj = L.genpsf(f,Nz=Nz,xsz=xsz,stagepos=f.res.stagepos)
#showpsf(f,p)

In [48]:
Im = f.res.I_model
Im = np.reshape(Im,(f.res.zernike_map.shape[-2],f.res.zernike_map.shape[-1],Nz,xsz,xsz))
f.res.I_model = Im

#### Generate spline model and save new PSF file

In [59]:
#coeff = L.gencspline(f.res,psfobj)
#f.locres.coeff = coeff
filename = resfile[:-3]+'_5D.h5'
L.writeh5file(filename,f.res,f.locres,f.rois)